In [1]:
import tensorflow as tf
import numpy as np

### construct Graph

In [4]:
hello=tf.constant("hello, world.")
a=tf.constant([3,2])
b=tf.constant([1,5])
# print(type(b))
c=tf.add(a,b)
# print(type(c))
print(c.get_shape())

(2,)


### run Graph in session 

In [4]:
with tf.Session() as sess:
    print(sess.run(hello))
    print(sess.run(c))
    print(c.eval())

b'hello, world.'
7
7


### another way to run sess

In [9]:
sess=tf.Session()
print(b.eval(session=sess))
sess.close()

4


## Graph as default

In [12]:
hi=tf.constant("HI")
print(hi.graph)
print(tf.get_default_graph())
# assert hi.graph is tf.get_default_graph()

with tf.Graph().as_default():
    hi2=tf.constant("hi2")
    print(tf.get_default_graph())
    print(hi2.graph)
    
with tf.Session() as sess:
    print(sess.run(hi))
    # print(sess.run(hi2)) # ERROR! is not an element of this graph.

b'HI'


In [16]:
# since FUNCTION is only compiled when CALL, so is default_graph is 
# the same as its CALLER
def func_graph(x):
    y=tf.add(x, x)
    print(y.graph)
    return y
a=tf.constant(2)
x=tf.constant(2)
y=func_graph(x)
with tf.Session() as sess:
    print(tf.get_default_graph())
    print(x.graph)
    print(y.graph)
    print(sess.run(y))

4


## specify graph in sess

In [24]:
# It's ok to run different Graph in a same sess. But you have to specify a Graph when init a Session.
with tf.Graph().as_default() as g1:
    a=tf.constant(1)
with tf.Graph().as_default() as g2:
    b=tf.constant(2)

with tf.Session(graph=g2) as sess:
    print(sess.run(b))

2


### variables

In [19]:
w=tf.Variable(initial_value=[[1,2,1],[1,3,4]],dtype=tf.float32, name='w')
x=tf.Variable(initial_value=[[1,1],[1,1],[1,1]],dtype=tf.float32, name='inputx')
y=tf.matmul(w,x)
z=tf.sigmoid(y)
print(z)
init_op=tf.global_variables_initializer()

Tensor("Sigmoid_1:0", shape=(2, 2), dtype=float32)


In [18]:
with tf.Session() as sess:
    sess.run(init_op)
    r=sess.run(z)
    r1=z.eval()
    print(r, r1)

[[ 0.98201376  0.98201376]
 [ 0.99966466  0.99966466]] [[ 0.98201376  0.98201376]
 [ 0.99966466  0.99966466]]


In [22]:
x1=tf.placeholder(tf.float32, shape=[2,3])
x2=tf.placeholder(tf.float32, shape=[3,2])
y=tf.matmul(x1,x2)
with tf.Session() as sess:
    m1=np.random.rand(2,3)
    m2=np.random.rand(3,2)
    print(sess.run(y, feed_dict={x1:m1, x2:m2}))

[[ 1.05303216  1.08981395]
 [ 1.53957582  1.75329125]]


## example linear regression

In [ ]:
import matplotlib.pyplot as plt

# Prepare train data
train_X = np.linspace(-1, 1, 100)
train_Y = 2 * train_X + np.random.randn(*train_X.shape) * 0.33 + 10

# Define the model
X = tf.placeholder("float")
Y = tf.placeholder("float")
w = tf.Variable(0.0, name="weight")
b = tf.Variable(0.0, name="bias")
loss = tf.square(Y - X*w - b)
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# Create session to run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    epoch = 1
    for i in range(10):
        for (x, y) in zip(train_X, train_Y):
            _, w_value, b_value = sess.run([train_op, w, b],feed_dict={X: x,Y: y})
        print("Epoch: {}, w: {}, b: {}".format(epoch, w_value, b_value))
        epoch += 1


#draw
plt.plot(train_X,train_Y,"+")
plt.plot(train_X,train_X.dot(w_value)+b_value)
plt.show()

## variable name & ops name

In [3]:
# tf.name_scope only for OPS name!
with tf.variable_scope("foo"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
assert v.name == "foo/v:0"
assert x.op.name == "foo/bar/add"

In [6]:
# tf.get_variable()  tf.Variable(). The former will check if variable name already exist.
# if exist && not reuse, will raise ERROR.
# one way to solve this is use SCOPE and set REUSE
def network(input):
    w=tf.get_variable(name="w", shape=[1,2],initializer=tf.ones_initializer())
    print(w.name)
    o=tf.add(w, input)
    return o

with tf.variable_scope('encode',reuse=True) as scope1:
    in1=tf.constant([1.0,1.0])
    in2=tf.constant([3.0,3])
    r1=network(in1)
    #scope1.reuse_variables()
    r2=network(in2)
    print(r1)

encode/w:0
encode/w:0
Tensor("encode_4/Add:0", shape=(1, 2), dtype=float32)


In [7]:
# REUSE don't work for tf.Variable, tf.Variable always make new variables.
with tf.variable_scope("scope1"):
    w1 = tf.get_variable("w1", shape=[])
    w2 = tf.Variable(0.0, name="w2")
with tf.variable_scope("scope1", reuse=True):
    w1_p = tf.get_variable("w1", shape=[])
    w2_p = tf.Variable(1.0, name="w2")

print(w1 is w1_p, w2 is w2_p)

True False


In [ ]:
v1=tf.Variable([1,2], name='v')
print(v1.name)
v2=tf.get_variable(name='v', shape=[2])
print(v2.name)

In [26]:
# use of tf.get_variable
a1=tf.get_variable(name='a1', shape=[])
a2=tf.get_variable(name='a2', shape=[2], initializer=tf.constant_initializer(5,5))
a3=tf.get_variable(name='a3', shape=[2,3], initializer=tf.random_normal_initializer(mean=0.,stddev=0.1))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(a1.eval())
    print(a2.eval())
    print(a3.eval())

[[ 0.09999033 -0.09345477  0.08982938]
 [ 0.13874331  0.09924894 -0.07598428]]


## Batch Normalization